# Set up

In [ ]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

In [ ]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools

In [ ]:
Device = 'cpu'

In [ ]:
# Device = 'cuda'

# Read data

In [ ]:
TrainData = pd.read_csv('Data/train3D.csv', delimiter=',')
TestData = pd.read_csv('Data/test3D.csv', delimiter=',')

In [ ]:
scaler = StandardScaler()
TrainData_standardized = pd.DataFrame(scaler.fit_transform(TrainData), columns=TrainData.columns).values

TestData_standardized = pd.DataFrame(scaler.fit_transform(TestData), columns=TestData.columns).values

X_train = TrainData_standardized[:,0:2]
Y_train = TrainData_standardized[:,-1]

X_test = TestData_standardized[:,0:2]
Y_test = TestData_standardized[:,-1]

In [ ]:
TrainData_log = np.log(TrainData).values
TestData_log = np.log(TestData).values

X_train = TrainData_log[:,0]
Y_train = TrainData_log[:,-1]

X_test = TestData_log[:,0]
Y_test = TestData_log[:,-1]

In [ ]:
train_x = torch.tensor(X_train, dtype=torch.float32).to(Device)
test_x = torch.tensor(X_test, dtype=torch.float32).to(Device)

train_y = torch.tensor(Y_train, dtype=torch.float32).to(Device)
test_y = torch.tensor(Y_test, dtype=torch.float32).to(Device)

# Training model

In [ ]:
LocalGP_models, LocalGP_likelihoods = Training.train_one_column_StandardGP(
    train_x, train_y, covar_type='RBF', lr=0.05, num_iterations=5000, patience=10, device=Device, use_amp=False
    )

# Prediction

In [ ]:
preds = Prediction.preds_for_one_model(LocalGP_models, LocalGP_likelihoods, test_x)

In [ ]:
preds = Prediction.preds_for_one_model(LocalGP_models, LocalGP_likelihoods, test_x).cpu().detach().numpy()

In [ ]:
np.mean((preds - test_y.numpy()) ** 2)

# Estimation

## Point estimation

In [ ]:
row_idx = 0

bounds = bound.get_bounds(train_x)

estimated_params, func_loss = Estimation.multi_start_estimation(LocalGP_models, LocalGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                attraction_threshold=0.1, repulsion_strength=0.5, device=Device)

In [ ]:
test_x[row_idx]

In [ ]:
estimated_params

## MCMC 

In [ ]:
# Only can run in .py

if __name__ == '__main__':
    row_idx = 0

    bounds = bound.get_bounds(train_x)

    mcmc_result = Estimation.run_mcmc(LocalGP_models, LocalGP_likelihoods, row_idx, test_y, bounds, 
                                                  num_sampling = 1000, warmup_step = 200, num_chains=1, device=Device,jit_compile=False)
    samples = mcmc_result.get_samples()
    print("theta 后验样本：", samples.get('theta'))
    print("sigma 后验样本：", samples.get('sigma'))


In [ ]:
posterior_samples_Normal = mcmc_result.get_samples()


true_values = test_x[row_idx, :].cpu().detach().numpy() if hasattr(test_x, "cpu") else test_x[row_idx, :]
point_estimations = None

num_params = len(posterior_samples_Normal)
fig, axes = plt.subplots(num_params, 1, figsize=(8, num_params * 3))
if num_params == 1:
    axes = [axes]

for i, (param_name, samples) in enumerate(posterior_samples_Normal.items()):
    ax = axes[i]
    samples_np = samples.cpu().detach().numpy() if samples.device.type != 'cpu' else samples.detach().numpy()
    sns.kdeplot(samples_np, ax=ax, color='blue')
    ax.set_title(f'Density of {param_name}')
    
    if true_values is not None and i < len(true_values):
        ax.axvline(true_values[i], color='red', linestyle='--', label='True Value')
    if point_estimations is not None and i < len(point_estimations):
        ax.axvline(point_estimations[i], color='green', linestyle='-.', label='Point Estimation')
    ax.legend()

plt.tight_layout()
plt.show()
